This notebook is aiming at prepare training data for first binary classifier of multi-step strategies. It also contains the part to kick off training jobs after data is prepared. (Training job will use "auto-split" mode to reserve validation data)

Prerequisite:

1) csv files containing extracted text files

2) A csv file prepared in Step2 notebook, containing label information corresponding to text files

Output:

1) Prepared training file for binary classifier

2) Kick off binary classification training job

###### This notebook should be executed after completing Step 1, 2, (2.5) notebooks.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import boto3

In [2]:
# Load prior prepared extracted text files
textract_df = pd.read_csv("Textract_output_train04.csv")
textract_df.rename(columns={'file_name' : 'filename'}, inplace=True)
textract_df

filename  page  \
0                 1003700000000530903_11159719.pdf     1   
1                 1003700000000530903_11159719.pdf     2   
2                 1003700000000530903_11159719.pdf     3   
3                 1003700000000530903_11159719.pdf     4   
4                 1003700000000530903_11159719.pdf     5   
...                                            ...   ...   
840163  OCR_33049_1003700000000697103_11103804.pdf    43   
840164  OCR_33049_1003700000000697103_11103804.pdf    44   
840165  OCR_33049_1003700000000697103_11103804.pdf    45   
840166  OCR_33049_1003700000000697103_11103804.pdf    46   
840167  OCR_33049_1003700000000697103_11103804.pdf    47   

                                                  content  
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...  
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...  
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...  
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...  
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...  
...                                                   ...  
840163  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  
840164  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840165  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840166  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840167  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  

[840168 rows x 3 columns]

In [3]:
# Change filename to same format 
textract_df['filename'] = textract_df['filename'].str.split('_').str[-2] + '_' + textract_df['filename'].str.split('_').str[-1]

In [4]:
# Load prior prepared reformat label file
label_df = pd.read_csv("batch3_label_revised.csv")
label_df

page             class                          filename  filename_cut
0         29   admission_order  1003700000000633142_11068501.pdf        633142
1         43  history_physical  1003700000000633142_11068501.pdf        633142
2         44  history_physical  1003700000000633142_11068501.pdf        633142
3         45  history_physical  1003700000000633142_11068501.pdf        633142
4         46  history_physical  1003700000000633142_11068501.pdf        633142
...      ...               ...                               ...           ...
357778    81  history_physical  1003700000000675009_11251999.pdf        675009
357779    82  history_physical  1003700000000675009_11251999.pdf        675009
357780    83  history_physical  1003700000000675009_11251999.pdf        675009
357781    84  history_physical  1003700000000675009_11251999.pdf        675009
357782    85  history_physical  1003700000000675009_11251999.pdf        675009

[357783 rows x 4 columns]

In [5]:
# Rename all 17 meaningful classes to "of_interest"
label_df['class'] = "of_interest"

In [7]:
# Check common file
common_files = set(textract_df.filename).intersection(label_df.filename)
common_files_train = common_files
len(common_files_train)

2378

In [8]:
# filter out uncommon file in text dataframe
textract_df_train = textract_df[textract_df.filename.isin(common_files_train)]
textract_df_train

filename  page  \
0       1003700000000530903_11159719.pdf     1   
1       1003700000000530903_11159719.pdf     2   
2       1003700000000530903_11159719.pdf     3   
3       1003700000000530903_11159719.pdf     4   
4       1003700000000530903_11159719.pdf     5   
...                                  ...   ...   
840163  1003700000000697103_11103804.pdf    43   
840164  1003700000000697103_11103804.pdf    44   
840165  1003700000000697103_11103804.pdf    45   
840166  1003700000000697103_11103804.pdf    46   
840167  1003700000000697103_11103804.pdf    47   

                                                  content  
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...  
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...  
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...  
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...  
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...  
...                                                   ...  
840163  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  
840164  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840165  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840166  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  
840167  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  

[838573 rows x 3 columns]

In [9]:
# filter out uncommon file in label dataframe
label_df_train = label_df[label_df.filename.isin(common_files_train)]
label_df_train

page        class                          filename  filename_cut
0         29  of_interest  1003700000000633142_11068501.pdf        633142
1         43  of_interest  1003700000000633142_11068501.pdf        633142
2         44  of_interest  1003700000000633142_11068501.pdf        633142
3         45  of_interest  1003700000000633142_11068501.pdf        633142
4         46  of_interest  1003700000000633142_11068501.pdf        633142
...      ...          ...                               ...           ...
357778    81  of_interest  1003700000000675009_11251999.pdf        675009
357779    82  of_interest  1003700000000675009_11251999.pdf        675009
357780    83  of_interest  1003700000000675009_11251999.pdf        675009
357781    84  of_interest  1003700000000675009_11251999.pdf        675009
357782    85  of_interest  1003700000000675009_11251999.pdf        675009

[356650 rows x 4 columns]

In [10]:
# merge text files with labels
merged_file_train = pd.merge(textract_df_train, label_df_train, how = "left", on = ["filename", "page"])
merged_file_train

filename  page  \
0       1003700000000530903_11159719.pdf     1   
1       1003700000000530903_11159719.pdf     2   
2       1003700000000530903_11159719.pdf     3   
3       1003700000000530903_11159719.pdf     4   
4       1003700000000530903_11159719.pdf     5   
...                                  ...   ...   
859387  1003700000000697103_11103804.pdf    43   
859388  1003700000000697103_11103804.pdf    44   
859389  1003700000000697103_11103804.pdf    45   
859390  1003700000000697103_11103804.pdf    46   
859391  1003700000000697103_11103804.pdf    47   

                                                  content        class  \
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...          NaN   
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...          NaN   
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...          NaN   
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...          NaN   
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...          NaN   
...                                                   ...          ...   
859387  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  of_interest   
859388  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  of_interest   
859389  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  of_interest   
859390  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  of_interest   
859391  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  of_interest   

        filename_cut  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                NaN  
...              ...  
859387      697103.0  
859388      697103.0  
859389      697103.0  
859390      697103.0  
859391      697103.0  

[859392 rows x 5 columns]

In [11]:
# Fill unlabelled data to "others" class
merged_file_train['class'] = merged_file_train['class'].fillna('others')
merged_file_train

filename  page  \
0       1003700000000530903_11159719.pdf     1   
1       1003700000000530903_11159719.pdf     2   
2       1003700000000530903_11159719.pdf     3   
3       1003700000000530903_11159719.pdf     4   
4       1003700000000530903_11159719.pdf     5   
...                                  ...   ...   
859387  1003700000000697103_11103804.pdf    43   
859388  1003700000000697103_11103804.pdf    44   
859389  1003700000000697103_11103804.pdf    45   
859390  1003700000000697103_11103804.pdf    46   
859391  1003700000000697103_11103804.pdf    47   

                                                  content        class  \
0       PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...       others   
1       UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...       others   
2       DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...       others   
3       UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...       others   
4       OF016785388 8105DOS 04/15/2018Commonwealth of ...       others   
...                                                   ...          ...   
859387  Patient: GABRY, KAREN11800 E. Twelve Mile Road...  of_interest   
859388  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  of_interest   
859389  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  of_interest   
859390  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  of_interest   
859391  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...  of_interest   

        filename_cut  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                NaN  
...              ...  
859387      697103.0  
859388      697103.0  
859389      697103.0  
859390      697103.0  
859391      697103.0  

[859392 rows x 5 columns]

In [12]:
# Since replacing 18 classes to binary classes, we will introduce duplicate records. We need to remove the duplicate records.
merged_file_train.drop_duplicates(inplace=True)

In [13]:
# Check distribution
merged_file_train['class'].value_counts()

others         501740
of_interest    334457
Name: class, dtype: int64

In [14]:
# Save merged dataframe with metainfo locally (will be used to organize data for later steps)
merged_file_train.to_csv('multistep-1-binary-training-with-metainfo-autosplit.csv', index=False)

In [15]:
# To prepare for training data, only select "class" and "content" columns
merged_file_train = merged_file_train[['class', 'content']]

In [16]:
# Save the perpared training data to local
merged_file_train.to_csv('multistep-1-binary-training-data-autosplit.csv', index=False, header=False)

In [17]:
# Check the perpared training data
merged_file_train

class                                            content
0            others  PATIENT NAME : BAKER, EDWINNA TLEXINGTON, KENT...
1            others  UKHealthCareBAKER, EDWINNADOS: 04/15/2018DOB: ...
2            others  DOS: 04/15/2018UKHealthCareBAKER, EDWINNADOB: ...
3            others  UK HealthCareBAKER, EDWINNADOS: 04/15/2018DOB:...
4            others  OF016785388 8105DOS 04/15/2018Commonwealth of ...
...             ...                                                ...
856940  of_interest  Patient: GABRY, KAREN11800 E. Twelve Mile Road...
856941  of_interest  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...
856942  of_interest  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...
856943  of_interest  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...
856944  of_interest  Patient:GABRY, KAREN11800 E. Twelve Mile RoadA...

[836197 rows x 2 columns]

In [19]:
# Specify region and client
REGION = 'us-east-1'
s3 = boto3.resource('s3')
comprehend = boto3.client('comprehend', region_name = REGION)

In [20]:
# upload training file to S3 (we're uploading to a folder that training pipeline is not monitoring, so we can manually kickoff jobs)
s3.Bucket('scio-saas-datastores').upload_file('multistep-1-binary-training-data-autosplit.csv', 'SaaS_workspace/aws_comprehend/inputs_manual/multistep-1-binary-training-data-autosplit.csv')

In [21]:
# Specify training parameters (classifier name has to be UNIQUE)
name = 'multistep-1-custom-classifier-binary-yy-autosplit'
language_code = 'en'
training_bucket = 'scio-saas-datastores'
training_key = 'SaaS_workspace/aws_comprehend/inputs_manual/multistep-1-binary-training-data-autosplit.csv'
data_access_role_arn = 'arn:aws:iam::097079433570:role/service-role/AmazonComprehendServiceRole-aws_scio_S3Datastore_servicewrite'

In [22]:
# Kick off training job
response = comprehend.create_document_classifier(
        DocumentClassifierName = name,
        LanguageCode = language_code,
        InputDataConfig = {
            "DataFormat": "COMPREHEND_CSV",
            "S3Uri":f"s3://{training_bucket}/{training_key}"
            },

        OutputDataConfig = {
            "S3Uri":"s3://scio-saas-datastores/SaaS_workspace/aws_comprehend/output/batch3-multistep",
            "KmsKeyId": "arn:aws:kms:us-east-1:097079433570:key/1c6d057c-0be3-40a6-b23c-5870a92b901f"
        },

        DataAccessRoleArn = data_access_role_arn,
        VolumeKmsKeyId = "arn:aws:kms:us-east-1:097079433570:key/1c6d057c-0be3-40a6-b23c-5870a92b901f",
        Mode = "MULTI_CLASS",

)